In [27]:
import joblib
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.tokenize import word_tokenize
import re
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import emoji

# Load the trained model and vectorizer
#model_path = "hard_voting_classifier.pkl"
#vectorizer_path = "vectorizer.pkl"

#loaded_model = joblib.load(model_path)
#loaded_vectorizer = joblib.load(vectorizer_path)

In [28]:
# 1. Load the model and vectorizer
model = joblib.load("hard_voting_classifier.pkl")
vectorizer = joblib.load("vectorizer.pkl")

In [29]:
with open("Indonesia_stopwords.txt", "r") as f:
    custom_stopwords = [word.strip() for word in f.readlines()]

In [30]:
def preprocess_data(text):

    # Case Folding
    def case_folding(text):
        return text.lower()


    # Sentence Normalization
    def normalize_sentence(text):
    # Translate emojis to their word representation
     text = emoji.demojize(text)
    
    # Remove URLs
     text = re.sub(r'http[s]?://\S+', '', text)
    
    # Remove numbers
     text = re.sub(r'\d+', '', text)
    
    # Remove non-alphanumeric characters except for spaces
     text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    
     return text
    
    # Word Tokenization
    tokens = word_tokenize(text)
    
    # Stop Words Removal
    tokens = [word for word in tokens if word not in custom_stopwords]
    
    # Stemming
    stemmer = StemmerFactory().create_stemmer()

    tokens = [stemmer.stem(word) for word in tokens]
    
    return ' '.join(tokens)

def predict_sentiment(text):
    # Preprocess the data
    preprocessed_text = preprocess_data(text)
    
    # Convert the preprocessed data into a vectorized format
    vectorized_text = vectorizer.transform([preprocessed_text])
    
    # Predict the sentiment using the model
    prediction = model.predict(vectorized_text)
    
    return "Positive" if prediction[0] == 1 else "Negative"


In [31]:
def predict_sentiment(text):
    preprocessed_text = preprocess_data(text)
    vectorized_text = vectorizer.transform([preprocessed_text])
    prediction = model.predict(vectorized_text)
    return prediction[0]  # Return the number directly instead of the string

unseen_data = ["wahh saya suka ini, tapi perlu ada fitur tambahan",
               "setelah diupdate, aplikasi ini sering crash di ponsel saya",
               "aplikasi ini sangat membantu saya dalam mengatur keuangan saya",]

predictions = [predict_sentiment(text) for text in unseen_data]
print(predictions)


[1, 0, 1]
